<a href="https://colab.research.google.com/github/JulioLaz/chatbot_spring_02/blob/main/Chatbot_Inteligente_Alura_Sprint2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [1]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
!pip install python-docx
from docx import Document
import csv
import nltk
nltk.download('punkt')

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Chatbot'

2023-09-28 01:56:43.836643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 11.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=8d4c3a4f013bb5045d441ae33eb74f0e5e6752af358556ac9fceca7cd74ff16d
  Stored in di

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


# 2. Importar verbos

In [2]:
import pickle

archivo_pickle_verbos = "/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle"
archivo_pickle_verbos_irregulares = "/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle"

# Importar la lista_verbos:
with open(archivo_pickle_verbos, "rb") as verbos:
        lista_verbos = pickle.load(verbos)

# Importar el diccionario:
with open(archivo_pickle_verbos_irregulares, "rb") as verbos_irregulares:
        lista_verbos_irregulares = pickle.load(verbos_irregulares)


print(lista_verbos)
print('---------------------------------')
print(lista_verbos_irregulares)

['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

#3. Tratamiento de datos

In [188]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
    max_similitud = 0.0
    for verbo in lista_verbos:
        similitud = jellyfish.jaro_similarity(palabra, verbo)
        if similitud > max_similitud:
            max_similitud = similitud
            palabra_encontrada = verbo

    if max_similitud >= 0.93:
        return palabra_encontrada, max_similitud
    else:
        return palabra, max_similitud

def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = re.sub(r"[^\w\s+\-*/]", '', texto)
  texto = " ".join(texto.split())
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(frase):
    terminaciones = ["es", "me", "as", "ste", "te"]
    palabras = frase.split()

    for i, palabra in enumerate(palabras):
        for terminacion in terminaciones:
            if palabra.endswith(terminacion) and len(palabra) > len(terminacion):
                palabras[i] = palabra[:-len(terminacion)] + "r"
                break

    nueva_frase = " ".join(palabras)
    return nueva_frase

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
  texto=tratamiento_texto(texto)
  #tokens: Es una lista vacia o no
  #texto: Es la pregunta del usuario
  if len(tokens)==0: #tokens está vacio, vamos a incluir tokens especiales
    #Si encuentras alguna palabra compuesta especial en el texto adiciona su token:
    #Si existe alguna: ['cientifico de datos', 'data scientist'] -> tokens.append('datascientist')
    #Si existe alguna: ['ciencia de datos', 'data science'] -> tokens.append('datascience')
    #Si existe alguna: ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo'] -> tokens.append('elprofealejo')

    if any(name in texto for name in ['cientifico de datos', 'data scientist']):
        tokens.append('datascientist')
    if any(name in texto for name in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo']):
        tokens.append('elprofealejo')
    if any(name in texto for name in ['ciencia de datos', 'data science']):
        tokens.append('datascience')
    # print('tokens: ', tokens)
  else: #tokens no está vacio, vamos a eliminar tokens irrelevantes
    elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: elementos_a_eliminar.append('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: elementos_a_eliminar.append('contar')
    elif 'hago' in texto and 'hacer' in tokens: elementos_a_eliminar.append('hacer')
    elif 'entiendes' in texto and 'entender' in tokens: elementos_a_eliminar.append('entender')
    elif 'sabes' in texto and 'saber' in tokens: elementos_a_eliminar.append('saber')
    #Ahora que tenemos nuestra blacklist(tokens irrelevantes) elimínalos de nuestra lista tokens
    tokens = [token for token in tokens if token not in elementos_a_eliminar]
    # print('elementos_a_eliminar: ',elementos_a_eliminar)
    # print(tokens)
  return tokens

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
    tokens = []
    tokens = revisar_tokens(texto, tokens)
    doc = nlp(texto)
    for t in doc:
        # Obtener el lemma
        lemma = lista_verbos_irregulares.get(t.text, t.lemma_)
        # print('lemma001: ',lemma)
        # lemma=lista_verbos_irregulares.get(t.text, t.lemma_.split()[0])

        # Verificar si lemma es una cadena no vacía
        if lemma and isinstance(lemma, str):
            lemma = re.sub(r'[^\w\s+\-*/]', '', lemma)

            if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
                if t.pos_ == 'VERB':
                    # print('lemma002: ',lemma)
                    lemma = reemplazar_terminacion(lemma)
                    # print('lemma003: ',lemma)
                    tokens.append(raiz(tratamiento_texto(lemma)))
                    # print('tokens: ',tokens)
                else:
                    tokens.append(tratamiento_texto(lemma))

    tokens = list(dict.fromkeys(tokens))
    tokens = list(filter(None, tokens))
    tokens = revisar_tokens(texto, tokens)
    tokens_str = str(tokens)
    return tokens_str


In [189]:
normalizar("profe alejo cientifico de datos")

"['datascientist', 'elprofealejo', 'profe', ('alejo', 0.8222222222222223), 'cientifico', 'dato']"

# 4. Cargar bases de documentos

In [190]:
#Importando bases de dialogo fluído
txt_folder_path = folder+'/dialogos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  estado = True
  for line in f.read().split('\n'):
    if estado:
      line_tratado = tratamiento_texto(line)
      lista_dialogos.append(line_tratado)
      lista_tipo_dialogo.append(lista_documentos[idx][:-4])
    else:
      lista_dialogos_respuesta.append(line)
    estado=not estado

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

#Importando bases csv
txt_folder_path = folder+'/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as csv_txt:
    csv_text = csv.reader(csv_txt)
    for fila in csv_text:
      if fila[-1]!='frase':
        documento_csv += fila[-1]

#Importando bases docx
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".docx")]
documento_docx = ''
for i in range(len(lista_documentos)):
  for t in Document(txt_folder_path+'/'+lista_documentos[i]).paragraphs:
    documento_docx += t.text.replace('*','\n\n*').replace('-','\n-')

#Importando bases txt
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
documento_txt = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as txt:
    txt_new = txt.read()
    for i in txt_new:
      documento_txt += i
# print('documento_csv: ',documento_csv)
# print('------------------------------------------------------------------------------')
# print('documento_docx: ', documento_docx)
# print('------------------------------------------------------------------------------')
# print('documento_txt: ', documento_txt)


In [214]:
# documento = documento_csv + documento_txt + documento_docx
documento = documento_txt+documento_docx+documento_csv
lista_frases = nltk.sent_tokenize(documento,'spanish')
lista_frases_normalizadas = [''.join(str(normalizar(x))) for x in lista_frases]

In [215]:
print(lista_frases)
print(lista_frases_normalizadas)

['La ciencia de datos es un campo académico interdisciplinario que utiliza estadística, computación científica, métodos, procesos, algoritmos y sistemas científicos para obtener (recolectar o extraer), tratar, analizar y presentar informes a partir de datos ruidosos, estructurados y no estructurados.', 'La ciencia de datos es multifacética y puede describirse como una ciencia, un paradigma de investigación, un método de investigación, una disciplina, un flujo de trabajo o una profesión.', 'La ciencia de datos integra el conocimiento del dominio de la aplicación subyacente (por ejemplo, ciencia económica, finanzas, medicina, ciencias naturales, tecnologías de la información) y es un "concepto consistente en unificar estadística, análisis de datos, informática y sus métodos relacionados" para "comprender y analizar fenómenos reales" con datos.', 'La ciencia de datos utiliza métodos, técnicas y teorías extraídas de muchos campos dentro del contexto de las matemáticas, las estadísticas, la

In [216]:
# df_dialogo.head(5)

###Guardar DF en drive:

In [217]:
import pandas as pd
import os

def guardar_dataframe(dataframe):
  carpeta = "/content/drive/MyDrive/Chatbot"
  nombre_archivo = "df_dialogo_spring_02.csv"

  if not os.path.exists(carpeta):
    os.makedirs(carpeta)
  archivo_csv = os.path.join(carpeta, nombre_archivo)
  dataframe.to_csv(archivo_csv, index=False)

# 5. Buscar respuesta del Chatbot

In [254]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import jellyfish

def dialogo(user_response):
    # Tratamiento de texto
    user_response = tratamiento_texto(user_response)  # Tratando el texto
    user_response = re.sub(r"[^\w\s]", '', user_response)  # Elimina signos de puntuación

    df = df_dialogo.copy()

    tfidf_vectorizer = TfidfVectorizer()

    for idx, row in df.iterrows():
        # Calcular la similitud de coseno usando TF-IDF
        tfidf_matrix = tfidf_vectorizer.fit_transform([user_response, row['dialogo']])
        cos_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

        df.at[idx, 'interseccion'] = len(set(user_response.split()) & set(row['dialogo'].split()))/len(user_response.split())
        df.at[idx, 'similarity'] = cos_sim[0][0]
        df.at[idx, 'jaro_winkler'] = jellyfish.jaro_winkler_similarity(user_response, row['dialogo'])

        # Calcular la probabilidad:
        df.at[idx, 'probabilidad'] = max(df.at[idx, 'interseccion'], df.at[idx, 'similarity'], df.at[idx, 'jaro_winkler'])

    df.sort_values(by=['probabilidad', 'jaro_winkler'], inplace=True, ascending=False)

    # Probabilidad máxima:
    probabilidad = round(df['probabilidad'].head(1).values[0],2)

    #Guardar df:
    guardar_dataframe(df)

    if probabilidad >= 0.93:
        print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
        respuesta = df['respuesta'].head(1).values[0]
    else:
        respuesta = ''
    return respuesta

#Cargar tu modelo entrenado aqui(recuerda siempre cargar el modelo y el vectorizer o tokenizer usado en el entrenamiento del modelo):
ruta_modelo = '/content/drive/MyDrive/Chatbot/modelo_transformers'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

#Función para dialogar utilizando el modelo de Machine Learning:
def clasificacion_modelo(pregunta):
  frase = normalizar(pregunta)
  frase = ' '.join(str(elemento) for elemento in frase)
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  with torch.no_grad():
      outputs = Modelo_TF(input_ids, attention_mask)

  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)
  etiquetas_decodificadas = etiquetas_predichas.tolist()

  diccionario = {3: 'Continuacion', 10: 'Nombre', 2: 'Contacto', 13: 'Saludos', 14: 'Sentimiento', 9: 'Identidad', 15: 'Usuario', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 0: 'Agradecimiento', 5: 'Edad', 4: 'Despedida', 11: 'Origen', 12: 'Otros', 7: 'Error', 8: 'Funcion'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = diccionario[llave_buscada]

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

# Función para devolver la respuesta de los documentos
def respuesta_documento(pregunta):
  pregunta = normalizar(pregunta)
  def contar_coincidencias(frase):
    return sum(1 for elemento in pregunta if elemento in frase)

  diccionario = {valor: posicion for posicion, valor in enumerate(lista_frases_normalizadas)}
  lista = sorted(list(diccionario.keys()), key=contar_coincidencias, reverse=True)[:6]

  lista.append(''.join(pregunta))

  TfidfVec = TfidfVectorizer(tokenizer=normalizar)
  tfidf = TfidfVec.fit_transform(lista)

  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = round(flat[-2],2)
  if req_tfidf>=0.22:
    print('Respuesta encontrada por el método TfidfVectorizer - Probabilidad:', req_tfidf)
    respuesta = lista_frases[diccionario[lista[idx]]]
  else:
    respuesta = ''
  return respuesta

# Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      preg=pregunta
      respuesta = respuesta_documento(preg)
      if respuesta != '':
        return respuesta
      else:
        return 'Respuesta no encontrada'

In [237]:
# df_new=pd.read_csv('/content/drive/MyDrive/Chatbot/df_dialogo_spring_02.csv')
# df_new.sample(5)

In [238]:
import warnings
warnings.filterwarnings("ignore")

# 6. Ejecutar Chatbot

In [239]:
pregunta="ciencia de datos profe alejo?"
print('tokens pregunta: ',normalizar(pregunta))
print("Respuesta:")
respuesta_documento(pregunta)

tokens pregunta:  ['elprofealejo', 'datascience', 'ciencia', 'dato', 'profe', 'alejo']
Respuesta:
Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.94


'La ciencia de datos integra el conocimiento del dominio de la aplicación subyacente (por ejemplo, ciencia económica, finanzas, medicina, ciencias naturales, tecnologías de la información) y es un "concepto consistente en unificar estadística, análisis de datos, informática y sus métodos relacionados" para "comprender y analizar fenómenos reales" con datos.'

In [240]:
from time import time

In [241]:
def conversar(pregunta):
  t0 = time()
  respuesta = respuesta_chatbot(pregunta)
  tf = round(time()-t0,1)
  print(f'Tiempo de respuesta: {tf} seg')
  print('-----------------------------------------------------------------')
  print('Conversación: ',pregunta)
  print('-----------------------------------------------------------------')
  print(respuesta)
  print('-----------------------------------------------------------------')

In [255]:
conversar('como estas?')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
Tiempo de respuesta: 6.0 seg
-----------------------------------------------------------------
Conversación:  como estas?
-----------------------------------------------------------------
Estoy bien, gracias por preguntar.
-----------------------------------------------------------------


In [256]:
conversar('cursos fundamento')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.82
Tiempo de respuesta: 4.7 seg
-----------------------------------------------------------------
Conversación:  cursos fundamento
-----------------------------------------------------------------
La ciencia de datos integra el conocimiento del dominio de la aplicación subyacente (por ejemplo, ciencia económica, finanzas, medicina, ciencias naturales, tecnologías de la información) y es un "concepto consistente en unificar estadística, análisis de datos, informática y sus métodos relacionados" para "comprender y analizar fenómenos reales" con datos.
-----------------------------------------------------------------


In [257]:
conversar('cuentame de la ciencia de datos profe alejo')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.93
Tiempo de respuesta: 5.4 seg
-----------------------------------------------------------------
Conversación:  cuentame de la ciencia de datos profe alejo
-----------------------------------------------------------------
El proceso que sigue un científico de datos para trabajar o resolver problemas con los datos se puede resumir en estos pasos: 

- Extraer datos, independientemente de la fuente y de su volumen
- Limpiar los datos, para eliminar lo que pueda sesgar los resultados
- Procesar los datos usando métodos estadísticos como inferencia estadística, modelos de regresión, pruebas de hipótesis, etc
- Diseñar experimentos adicionales en caso de ser necesario
- Crear visualizaciones gráficas de los datos relevantes de la investigación.
-----------------------------------------------------------------


In [258]:
conversar("de que se trata la ciencia de datos y el profe alejo")

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.94
Tiempo de respuesta: 6.8 seg
-----------------------------------------------------------------
Conversación:  de que se trata la ciencia de datos y el profe alejo
-----------------------------------------------------------------
El proceso que sigue un científico de datos para trabajar o resolver problemas con los datos se puede resumir en estos pasos: 

- Extraer datos, independientemente de la fuente y de su volumen
- Limpiar los datos, para eliminar lo que pueda sesgar los resultados
- Procesar los datos usando métodos estadísticos como inferencia estadística, modelos de regresión, pruebas de hipótesis, etc
- Diseñar experimentos adicionales en caso de ser necesario
- Crear visualizaciones gráficas de los datos relevantes de la investigación.
-----------------------------------------------------------------


In [259]:
conversar('como te llamas?')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0
Tiempo de respuesta: 3.9 seg
-----------------------------------------------------------------
Conversación:  como te llamas?
-----------------------------------------------------------------
Me llamo X-Bot hijo de Madre. ¿Y tú?
-----------------------------------------------------------------


In [260]:
conversar('como te llamas amigo?')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  0.94
Tiempo de respuesta: 3.9 seg
-----------------------------------------------------------------
Conversación:  como te llamas amigo?
-----------------------------------------------------------------
Me llamo X-Bot hijo de Madre, ¡el chatbot más guay del barrio! ¿Y tú?
-----------------------------------------------------------------


In [261]:
conversar('que es un cientifico de datos?')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.84
Tiempo de respuesta: 6.9 seg
-----------------------------------------------------------------
Conversación:  que es un cientifico de datos?
-----------------------------------------------------------------
Un científico de datos es el profesional que mediante la escritura y aplicación de código de programación y conocimientos en estadística trabaja en la recolección de datos, la limpieza de datos, la exploración de datos, la modelación de datos, visualización de datos, la implementación de soluciones de aprendizaje automático y en la interpretación de resultados.
-----------------------------------------------------------------


In [262]:
conversar('Cómo estás amigo?')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.9
Tiempo de respuesta: 5.0 seg
-----------------------------------------------------------------
Conversación:  Cómo estás amigo?
-----------------------------------------------------------------
La ciencia de datos es diferente de la informática, la estadística y la ciencia de la información, el ganador del premio Turing, Jim Gray, imaginó la ciencia de datos como un "cuarto paradigma" de la ciencia (empírico, teórico, computacional y ahora basado en datos) y afirmó que "todo sobre la ciencia está cambiando debido al impacto de la tecnología de la información" y la avalancha de datos.
-----------------------------------------------------------------


In [263]:
conversar('La ciencia de datos es')

Respuesta encontrada por el modelo Transformers - tipo: Funcion
Tiempo de respuesta: 4.6 seg
-----------------------------------------------------------------
Conversación:  La ciencia de datos es
-----------------------------------------------------------------
Estoy familiarizado con una amplia gama de herramientas populares como Python, R, SQL, TensorFlow, scikit-learn, y muchas más. Cuéntame qué necesitas y te ayudaré.
-----------------------------------------------------------------


In [264]:
conversar('La ciencia de datos es')

Respuesta encontrada por el modelo Transformers - tipo: Funcion
Tiempo de respuesta: 6.7 seg
-----------------------------------------------------------------
Conversación:  La ciencia de datos es
-----------------------------------------------------------------
Estoy familiarizado con una amplia gama de herramientas populares como Python, R, SQL, TensorFlow, scikit-learn, y muchas más. Cuéntame qué necesitas y te ayudaré.
-----------------------------------------------------------------


In [265]:
conversar('cientifico de datos?')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.76
Tiempo de respuesta: 4.6 seg
-----------------------------------------------------------------
Conversación:  cientifico de datos?
-----------------------------------------------------------------
Un científico de datos es el profesional que mediante la escritura y aplicación de código de programación y conocimientos en estadística trabaja en la recolección de datos, la limpieza de datos, la exploración de datos, la modelación de datos, visualización de datos, la implementación de soluciones de aprendizaje automático y en la interpretación de resultados.
-----------------------------------------------------------------


In [266]:
conversar('que cursos hay')

Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.8
Tiempo de respuesta: 4.8 seg
-----------------------------------------------------------------
Conversación:  que cursos hay
-----------------------------------------------------------------
La ciencia de datos ha resultado para muchos una disciplina de reciente creación, pero en la realidad este concepto lo utilizó por primera vez el científico danés Peter Naur en la década de los sesenta como sustituto de las ciencias computacionales.
-----------------------------------------------------------------
